In [4]:
import requests
import pandas as pd
import pyspark

if not sc:
    sc = pyspark.SparkContext()

type(sc)

pyspark.context.SparkContext

# Objectifs :

   - construire un dataframe pandas qui regroupe l'ensemble des données que sur lesquelles on veut lancer notre apprentissage
     - données statiques additionnelles à ajouter:
        1 coefficient d'isolation (nombre de stations accessible depuis la station dans un rayon donné raisonnable 5km ?)
        associer la station météo la plus proche de la station vélib
        
$e^{i\pi} + 1 = 0$

$$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$$
       



# recupération des stations velibs de paris

In [21]:
urlParisStations = "https://api.jcdecaux.com/vls/v1/stations"
jcdKey = "c7c379f39e07e619be9663cfce805b8b266dce1a"
# https://api.jcdecaux.com/vls/v1/stations?contract={contract_name} 
stations_json = requests.get(url = urlParisStations, params={"contract":"Paris","apiKey":jcdKey}).text


In [24]:
df_stations = pd.read_json(stations_json,precise_float=True)\
                .drop("contract_name",axis=1)\
                .set_index("number")
df_stations.head()

,address,available_bike_stands,available_bikes,banking,bike_stands,bonus,last_update,name,position,status
number,,,,,,,,,,
31705,RUE DES CHAMPEAUX (PRES DE LA GARE ROUTIERE) -...,48,2,True,50,True,1489489741000,31705 - CHAMPEAUX (BAGNOLET),"{'lat': 48.8645278209514, 'lng': 2.41617072442...",OPEN
10042,52 RUE D'ENGHIEN / ANGLE RUE DU FAUBOURG POISS...,0,33,True,33,False,1489489649000,10042 - POISSONNIÈRE - ENGHIEN,"{'lat': 48.87242006305313, 'lng': 2.3483952362...",OPEN
8020,74 BOULEVARD DES BATIGNOLLES - 75008 PARIS,32,12,True,44,True,1489489725000,08020 - METRO ROME,"{'lat': 48.882148945631904, 'lng': 2.319860054...",OPEN
1022,37 RUE CASANOVA - 75001 PARIS,1,36,True,37,False,1489489512000,01022 - RUE DE LA PAIX,"{'lat': 48.8682170167744, 'lng': 2.33049351139...",OPEN
35014,139 AVENUE JEAN LOLIVE / MAIL CHARLES DE GAULL...,0,18,True,25,False,1489489725000,35014 - DE GAULLE (PANTIN),"{'lat': 48.893268664697416, 'lng': 2.412715733...",OPEN


# Récupération des stations météo de paris et alentours

source openWeather api:

https://api.openweathermap.org/data/2.5/weather?q={city name}

https://api.openweathermap.org/data/2.5/weather?q={city name},{country code}

https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}

apiKey : 679875fddff53d1517c9f4acda29535e

In [54]:
first_station = df_stations.head(1).position.values[0]
first_station

{'lat': 48.8645278209514, 'lng': 2.416170724425901}

In [69]:
df_stations.
#pd.DataFrame(index=df_stations.index.values(),data=df_stations.position.values()['lat'])

number
31705    {'lat': 48.8645278209514, 'lng': 2.41617072442...
10042    {'lat': 48.87242006305313, 'lng': 2.3483952362...
8020     {'lat': 48.882148945631904, 'lng': 2.319860054...
1022     {'lat': 48.8682170167744, 'lng': 2.33049351139...
35014    {'lat': 48.893268664697416, 'lng': 2.412715733...
20040    {'lat': 48.870393671603765, 'lng': 2.384222472...
28002                {'lat': 48.884478, 'lng': 2.24772065}
15111    {'lat': 48.834658862792736, 'lng': 2.295789480...
12124    {'lat': 48.839041145908176, 'lng': 2.437655339...
9021     {'lat': 48.877872278317966, 'lng': 2.337446804...
7007     {'lat': 48.85971010443503, 'lng': 2.3258090246...
20011    {'lat': 48.85550135398888, 'lng': 2.4051685206...
33006    {'lat': 48.9101436710038, 'lng': 2.38505201824...
5005     {'lat': 48.844730256132095, 'lng': 2.341923944...
11029    {'lat': 48.8666176586814, 'lng': 2.38301344041...
4020     {'lat': 48.86008832610481, 'lng': 2.3501483359...
7012     {'lat': 48.854268334051035, 'lng': 2.319